In [1]:
import numpy as np

In [2]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from lightning.pytorch import Trainer

/home/demancum/miniconda3/envs/cocola/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  

In [4]:
from contrastive_model.contrastive_model import CoCola
from data import (
musdb_contrastive_preprocessed,
moisesdb_contrastive_preprocessed,
slakh2100_contrastive_preprocessed,
coco_chorales_contrastive_preprocessed
)


In [23]:
CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/k5llcdm8/checkpoints/epoch=21-step=21406.ckpt' #bumbling-breeze-34 bs=512 chunl_dur=2s
#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/r71kxj7r/checkpoints/epoch=36-step=29119.ckpt' #good-wave-24 bs=256 chunl_dur=5s
#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/shov44gs/checkpoints/epoch=24-step=19900.ckpt' #golden-dew-25 bs=128 chunl_dur=10s

#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/zr304efn/checkpoints/epoch=23-step=17448.ckpt' #flowing-sound-26 bs=96 chunl_dur=15s

#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/8jtiowyz/checkpoints/epoch=46-step=38963.ckpt' #jumping-breeze-27 bs=64 chunl_dur=20s


model = CoCola.load_from_checkpoint(CHECKPOINT)
trainer = Trainer()


/home/demancum/miniconda3/envs/cocola/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/demancum/miniconda3/envs/cocola/lib/python3.11 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## MOISESDB

In [13]:
dataset = moisesdb_contrastive_preprocessed.get_dataset(
    chunk_duration=2,
    positive_noise=0,
    generate_submixtures=True,
    transform=None
    )

_, _, test_dataset = random_split(
    dataset=dataset,
    lengths=[0.8, 0.1, 0.1],
    generator=torch.Generator().manual_seed(42)
    )

test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )


INFO:Found original dataset at /disk1/demancum/moisesdb/moisesdb_v0.1.
INFO:Found preprocessed dataset at /disk1/demancum/moisesdb/preprocessed2.


In [14]:
trainer.test(model=model, dataloaders=test_dataloader)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Testing DataLoader 0: 100%|██████████| 2592/2592 [00:47<00:00, 54.18it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8858024691358025     │
│         test_loss         │     21.09698486328125     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 21.09698486328125, 'test_accuracy': 0.8858024691358025}]

## Reults MOISESDB

bumbling-breeze-34 bs=512 chunl_dur=2s test on moisesdb bs=512 [{'test_loss': 30.519054412841797, 'test_accuracy': 0.396484375}]

bumbling-breeze-34 bs=512 chunl_dur=2s test on moisesdb bs=64 [{'test_loss': 29.30805206298828, 'test_accuracy': 0.5985725308641975}]

bumbling-breeze-34 bs=512 chunl_dur=2s test on moisesdb bs=2 [{'test_loss': 25.239595413208008, 'test_accuracy': 0.8775077160493827}]

---

good-wave-24 bs=256 chunl_dur=5s, test on moisesdb bs=256 [{'test_loss': 2.751087188720703, 'test_accuracy': 0.41650390625}] 

good-wave-24 bs=256 chunl_dur=5s, test on moisesdb bs=64 [{'test_loss': 1.7366595268249512, 'test_accuracy': 0.5849609375}]

good-wave-24 bs=256 chunl_dur=5s, test on moisesdb bs=2 [{'test_loss': 0.1633053719997406, 'test_accuracy': 0.9415148609779481}]

---

golden-dew-25 bs=128 chunk_dur=10s test on moisesdb bs=128 [{'test_loss': 14.337109565734863, 'test_accuracy': 0.35379464285714285}]

golden-dew-25 bs=128 chunk_dur=10s test on moisesdb bs=64 [{'test_loss': 13.74421501159668, 'test_accuracy': 0.4654017857142857}]

golden-dew-25 bs=128 chunk_dur=10s test on moisesdb bs=2 [{'test_loss': 10.538426399230957, 'test_accuracy': 0.8828451882845187}]

---
flowing-sound-26 bs=96 chunl_dur=15s test on moisesdb bs=96 [{'test_loss': 3.2646923065185547, 'test_accuracy': 0.24702380952380953}]

flowing-sound-26 bs=96 chunl_dur=15s test on moisesdb bs=64 [{'test_loss': 2.8988101482391357, 'test_accuracy': 0.28267045454545453}]

flowing-sound-26 bs=96 chunl_dur=15s test on moisesdb bs=2 [{'test_loss': 0.29432547092437744, 'test_accuracy': 0.8728813559322034}]

---

jumping-breeze-27 bs=64 chunl_dur=20s test on moisesdb bs=64 [{'test_loss': 3.286912441253662, 'test_accuracy': 0.38671875}]

jumping-breeze-27 bs=64 chunl_dur=20s test on moisesdb bs=2 [{'test_loss': 0.5389121770858765, 'test_accuracy': 0.8936567164179104}]

---




## slakh2100

In [17]:
test_dataset = slakh2100_contrastive_preprocessed.get_dataset(
    split="test",
    chunk_duration=2,
    positive_noise=0,
    generate_submixtures=True,
    transform=None
    )

test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )

trainer.test(model=model, dataloaders=test_dataloader)

INFO:Found original dataset split test at /disk1/demancum/slakh2100_contrastive/original/slakh2100_redux_16k/test.
INFO:Found preprocessed dataset split test at /disk1/demancum/slakh2100_contrastive/preprocessed2/test.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Testing DataLoader 0: 100%|██████████| 19810/19810 [05:56<00:00, 55.54it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9223624432104998     │
│         test_loss         │    1.0304263830184937     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.0304263830184937, 'test_accuracy': 0.9223624432104998}]

## Results slakh2100

bumbling-breeze-34 bs=512 chunk_dur=2s test on slakh2100 bs=64 [{'test_loss': 2.6327898502349854, 'test_accuracy': 0.5836530694668821}]

bumbling-breeze-34 bs=512 chunk_dur=2s test on slakh2100 bs=2 [{'test_loss': 1.0304263830184937, 'test_accuracy': 0.9223624432104998}]

--- 

## COCOCHORALES

In [24]:
test_dataset = coco_chorales_contrastive_preprocessed.get_dataset(
    split="test",
    ensemble="*",
    chunk_duration=2,
    positive_noise=0,
    generate_submixtures=True,
    transform=None
    )
    
test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )


INFO:Found original dataset split test at /disk1/demancum/coco_chorales_contrastive/original/test.
INFO:Found preprocessed dataset split test at /disk1/demancum/coco_chorales_contrastive/preprocessed2/test.


In [25]:
trainer.test(model=model, dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Testing DataLoader 0:  38%|███▊      | 8680/22630 [02:38<04:13, 54.92it/s]

## Results COCOCHORALES

bumbling-breeze-34 bs=512 chunk_dur=2s test on COCOCHORALES bs=64 

bumbling-breeze-34 bs=512 chunk_dur=2s test on COCOCHORALES bs=2 